In [59]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

## Dataset

In [82]:
path = r'C:\Users\je7560\Downloads\matheus\kinematics-robotics\src\data\ready\dataset_10mm.csv'
df = pd.read_csv(path)
df.head()

,Unnamed: 0,theta_1,theta_2,theta_3,theta_4,theta_5,p_x,p_y,p_z,Roll,Pich,Yaw
0,0,0,0,12,0,0,53.213314,0.0,2.515179,3.141593,-1.361357,0.0
1,1,0,12,12,0,0,53.606657,0.0,13.742410,3.141593,-1.570796,0.0
2,2,0,12,24,0,0,52.819970,0.0,6.257590,3.141593,-1.361357,0.0
3,3,0,12,24,12,0,53.213314,0.0,10.000000,3.141593,-1.570796,0.0
4,4,0,12,36,12,0,51.657132,0.0,2.678740,3.141593,-1.361357,0.0


### Atributos e saídas

In [122]:
y = df.iloc[:,3:4]
x = df.iloc[:,6:12]

In [123]:
print(f"Tamanho do dataset: {y.shape}")
y.head()

Tamanho do dataset: (6891, 1)


,theta_3
0,12
1,12
2,24
3,24
4,36


In [124]:
x.head()

,p_x,p_y,p_z,Roll,Pich,Yaw
0,53.213314,0.0,2.515179,3.141593,-1.361357,0.0
1,53.606657,0.0,13.742410,3.141593,-1.570796,0.0
2,52.819970,0.0,6.257590,3.141593,-1.361357,0.0
3,53.213314,0.0,10.000000,3.141593,-1.570796,0.0
4,51.657132,0.0,2.678740,3.141593,-1.361357,0.0


### Treino e Teste

In [125]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [116]:
scaler = StandardScaler()
Xtrain = scaler.fit_transform(X_train)
Ytrain = scaler.fit_transform(y_train)

## Modelos

In [ ]:
def modelo(hidden):
    func_act = 'tanh'
    model = MLPRegressor(hidden_layer_sizes=hidden,
                        activation=func_act)

In [126]:
# Definir os parâmetros
taxa_aprendizado = 0.01
numero_epocas = 1000
erro_adm = 0.01
fator_momento = 0.9
validacao = 0.1

In [129]:
model = MLPRegressor(hidden_layer_sizes=(100, 200), #validação
                    activation='tanh',  # Função de ativação da camada oculta
                    random_state=42)

### Grid Search

In [24]:
def grid_search(X_train, y_train, param_grid, model):
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3,scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    #.ravel()

    # Avaliação do melhor hiperparâmetro
    melhor_modelo = grid_search.best_estimator_
    y_pred = melhor_modelo.predict(X_train)
    mse = mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)

    print("Melhores hiperparâmetros:", grid_search.best_params_)
    print(f"\nMSE nos dados completos: {mse}")
    print(f"R² nos dados completos: {r2}")

    # Extrair os resultados da validação cruzada
    resultados = pd.DataFrame(grid_search.cv_results_)

    colunas_interesse = ['param_hidden_layer_sizes', 'mean_test_score', 'std_test_score']
    resultados_interesse = resultados[colunas_interesse]

    print("Avaliação do MSE para cada hiperparâmetro testado:")
    print(resultados_interesse)
    return resultados

### Modelo

In [130]:
# Treinando o modelo
model.fit(Xtrain, Ytrain)

# Fazendo previsões
y_pred = model.predict(Xtrain)

# Avaliando o modelo
mse = mean_squared_error(Ytrain, y_pred)
print(mse)
#print(f"{Ytrain}  {y_pred}")

0.008671714073118121


## Métricas

In [ ]:
from sklearn.metrics import mean_squared_error

def predict_mse(model,X_test,Y_test):
    y_predic = model.predict(X_test)
    return mean_squared_error(Y_test, y_predic,multioutput='raw_values')